In [1]:
!which python

/Users/darrenkwon/statistics-and-data-driven/.venv/bin/python


In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pylab as plt

In [3]:
gss_df = pd.read_csv("../../data/gss_bayes.csv", index_col=0)
gss_df.head(2)

,year,age,sex,polviews,partyid,indus10
caseid,,,,,,
1,1974,21.0,1,4.0,2.0,4970.0
2,1974,41.0,1,5.0,0.0,9160.0


## 확률  

boolean series면 이거 평균 내면 그게 확률이잖아!

In [4]:
BANKER_INDUS10 = 6870
banker_ser = gss_df["indus10"] == BANKER_INDUS10

banker_ser.sum() # 은행원의 숫자
banker_ser.mean() # 은행원의 비율 

0.014769730168391155

In [7]:
def prob(A):
    """A should be a boolean series or array."""
    return A.mean()

In [8]:
prob(banker_ser) # 은행원의 비율. 0.014

gss_df["sex"].value_counts() # only 1 and 2
female_ser = gss_df["sex"] == 2
prob(female_ser) # 여성 비율이 0.53 정도

liberal_ser = gss_df["polviews"] <= 3
prob(liberal_ser) # 0.27

democrat_ser = gss_df["partyid"] <= 1
prob(democrat_ser) # 0.366

0.3662609048488537

In [9]:
# 은행원이고 민주당일 확률 (AND 조건. P(A and B))
prob(banker_ser & democrat_ser) # 0.004. 거의 0.4%

0.004686548995739501

## 조건부 확률

$P(\theta \mid D)$

- boolean 형의 series일 때, $\theta[D]$

In [10]:
# liberal 성향을 가진 사람을 한 명 골랐을 때 민주당원일 확률
prob(democrat_ser[liberal_ser]) # 0.52 

# 은행원을 한 명 골랐을 때 그게 여자일 확률
prob(female_ser[banker_ser]) # 77%

def conditional(proposition, given):
    """proposition, given should be boolean series or array."""
    return prob(proposition[given])

conditional(liberal_ser, female_ser) # 27%

0.27581004111500884

In [13]:
# 민주 성향이고 민주당원인 사람을 뽑았을 때 여성일 확률
conditional(female_ser, given=liberal_ser & democrat_ser) # 0.57

# 은행원일 때, 자유 성향인 여성일 확률
conditional(liberal_ser & female_ser, given=banker_ser)

0.17307692307692307

## bayes's theorem

- $ P(A \mid B) = P(A) \frac{P(B \mid A)}{P(B)} $  
- 사후확률 = 사전확률 * likelihood(우도) / evidence

### 전체 확률의 법칙


- $B_1, B_2$가 전체 포괄적이며 상호 배제적일 때(MECE), 
  - $P(A) = P(B_1 \cap A) + P(B_2 \cap A)$
  - 여기서 $P(A) = P(B_1)P(A \mid B_1) + P(B_2)P(A \mid B_2) $
  - 최종적으로 다음과 같다. $P(A) = \sum{P(B_i)P(A \mid B_i)}$

In [30]:
# P(은행) = P(은행 and 남성) + P(은행 and 여성) 일 것이란 계산.
# 이는 모든 성이 남성, 여성으로 MECE하기 때문

prob(banker_ser), prob(banker_ser & (gss_df["sex"] == 1)) + prob(banker_ser & (gss_df["sex"] == 2))

(0.014769730168391155, 0.014769730168391155)

In [34]:
B = gss_df["polviews"]
gss_df["polviews"].value_counts().sort_index()

1.0     1442
2.0     5808
3.0     6243
4.0    18943
5.0     7940
6.0     7319
7.0     1595
Name: polviews, dtype: int64

In [38]:
NEUTRAL = 4

# P(은행원 and 중도) = P(중도) * P(은행원 | 중도)
prob(B == NEUTRAL) * conditional(banker_ser, given=(B == NEUTRAL))

# 전체확률의 법칙 적용. 정치 성향 1 ~ 7에 대해
sum(prob(B == i) * conditional(banker_ser, given=(B == i)) for i in range(1, 8))

0.014769730168391157